In [1]:
"""
Name: movement.ipynb
Authors: Stephan Meighen-Berger, Martina Karl
Example how to create a video of the simulation
Requires mpl_toolkits and imageio
"""

'\nName: movement.ipynb\nAuthors: Stephan Meighen-Berger, Martina Karl\nExample how to create a video of the simulation\nRequires mpl_toolkits and imageio\n'

In [2]:
# General imports
import numpy as np
import matplotlib.pyplot as plt
import sys
from tqdm import tqdm
import matplotlib.colors as colors
import imageio

In [3]:
# Adding path to module
sys.path.append("../")

In [4]:
# picture path
PICS = '../pics/'

In [5]:
# Module imports
from fourth_day import Fourth_Day as FD
from fourth_day import config

In [6]:
# Scenario
config['scenario']['population size'] = 1
config['scenario']['duration'] = 1000
config['scenario']['organism movement'] = False
config['scenario']['exclusion'] = True
config['scenario']['injection']['rate'] = 1e-2
config['scenario']['injection']['y range'] = [5., 15.]
config['scenario']['light prop'] = {
            "switch": True,
            "x_pos": 5.,
            "y_pos": 10.,
        }
config['scenario']['detector'] = {
    "switch": True,
    "type": "PMTSpec",
    "response": True,
    "acceptance": "Flat",
    "mean detection prob": 0.5
}
# Organisms 
config['organisms']['emission fraction'] = 0.1
config['organisms']['alpha'] = 2.
config['organisms']['photon yield'] = 1e6
# Geometry
config['geometry']['volume'] = {
    'function': 'rectangle',
    'x_length': 40.,
    'y_length': 20.,
    'offset': None,
}
config['geometry']['observation'] = {
    'function': 'rectangle',
    'x_length': 40.,
    'y_length': 12.,
    "offset": np.array([0., 4.]),
}
config['geometry']["exclusion"] = {
    "function": "sphere",
    "radius": 0.3,
    "x_pos": 5.,
    "y_pos": 10.,
}
# Water
config['water']['model']['name'] = 'custom' # 'potential cylinder'
config['water']['model']['directory'] = "../data/current/cyclic/"
config['water']['model']['time step'] = 50
config['advanced']['starting step'] = 575

In [7]:
statistics = []
wavelengths = []
measured = []
seeds = [10, 1337]
for seed in seeds:
    # General
    config["general"]["random state seed"] = seed
    # Creating a fourth_day object
    fd = FD()
    # Launching solver
    fd.sim()
    statistics.append(fd.statistics)
    wavelengths.append(fd.wavelengths)
    measured.append(fd.measured)

In [8]:
# Plotting standards
std_size = 12
fontsize = 12.
lw=3.
h_length=1.5

In [9]:
# Offset
off_set = 500

In [23]:
# Random noise
noise = []
for i, _ in enumerate(fd.measured.keys()):
    noise.append(np.abs(np.random.normal(2e2, 1e2, len(fd.t))))
noise = np.array(noise)
x_grid = np.linspace(0., 40., 300)
y_grid = np.linspace(4., 16., 300)
X, Y = np.meshgrid(x_grid, y_grid)
all_points = []
for i in range(0, len(x_grid)):
    all_points.append(np.vstack((X[i], Y[i])).T)
for i, _ in tqdm(enumerate(fd.t[off_set:])):
    q = i + off_set
    figure, ((ax, ax3), (ax1, ax4)) = plt.subplots(2, 2, figsize=(std_size, std_size * 6. / 8.))
    # The water current
    x_vel = []
    y_vel = []
    grad = []
    for j in range(0, len(x_grid)):
        x_vel_tmp, y_vel_tmp, grad_tmp = fd._current.velocities(all_points[j], q + config['advanced']['starting step'])
        x_vel.append(x_vel_tmp)
        y_vel.append(y_vel_tmp)
        grad.append(grad_tmp)
    ax.quiver(x_grid, y_grid, x_vel, y_vel, grad)
    # The emitters
    emission_mask = statistics[0][q].loc[:, 'is_emitting']
    ax.scatter(statistics[0][q].loc[~emission_mask, 'pos_x'].values,
               statistics[0][q].loc[~emission_mask, 'pos_y'].values,
               color='b',
               s=10.
    )
    ax.scatter(statistics[0][q].loc[emission_mask, 'pos_x'].values,
               statistics[0][q].loc[emission_mask, 'pos_y'].values,
               color='r',
               s=10.
    )
    # Exclusion
    # theta goes from 0 to 2pi
    theta = np.linspace(0, 2*np.pi, 50)
    # the radius of the circle
    r = config['geometry']['exclusion']['radius']
    # compute x1 and x2
    x1 = r*np.cos(theta)
    x2 = r*np.sin(theta)
    ax.plot(x1 + config['geometry']['exclusion']['x_pos'], x2 + config['geometry']['exclusion']['y_pos'],
            lw=lw, color='k')
    # Additional options
    ax.set_xlabel(r'$X\;[\mathrm{m}]$', fontsize=fontsize)
    ax.set_ylabel(r'$Y\;[\mathrm{m}]$', fontsize=fontsize)
    ax.tick_params(axis = 'both', which = 'major', labelsize=fontsize)
    ax.tick_params(axis = 'both', which = 'minor', labelsize=fontsize)
    ax.title=ax.set_title('Population t = %.f s' %(fd.t[q] / 5.), fontsize=fontsize)
    ax.set_xlim(0., 40.)
    ax.set_ylim(4., 16.)
    # -----------------------------------------------------------------------------------------------------
    # The light emission curve
    for j, detector in enumerate(fd.measured.keys()):
        ax1.plot((fd.t / 5.)[:q+1],
             measured[0][detector].values[:q+1] + noise[j, :q+1],
             lw=lw, label=detector)
    #ax1.set_ylim(1e6, 1e11)
    ax1.set_xscale('linear')
    ax1.set_yscale('log')
    ax1.set_xlabel(r'$t\;[$s$]$', fontsize=fontsize)
    ax1.set_ylabel(r'$Unweighted\;Photons$', fontsize=fontsize)
    ax1.tick_params(axis = 'both', which = 'major', labelsize=fontsize, direction='in')
    ax1.tick_params(axis = 'both', which = 'minor', labelsize=fontsize, direction='in')
    ax1.grid(True)
    h, l = ax1.get_legend_handles_labels()
    lgd1 = ax1.legend(h,l, loc=9, bbox_to_anchor=(0.9, +1.),
                      ncol=1, fontsize=fontsize, handlelength=h_length,
                      fancybox=True, frameon=True)
    ax1.set_xlim(fd.t[off_set] / 5., max(fd.t / 5.))
    ax1.set_ylim(2e1,1e5)
    # ----------------------------------------------------------------------------------------------------
    # Water current
    ax3.quiver(x_grid, y_grid, x_vel, y_vel, grad)
    # Emitters
    emission_mask = statistics[1][q].loc[:, 'is_emitting']
    ax3.scatter(statistics[1][q].loc[~emission_mask, 'pos_x'].values,
               statistics[1][q].loc[~emission_mask, 'pos_y'].values,
               color='b',
               s=10.
    )
    ax3.scatter(statistics[1][q].loc[emission_mask, 'pos_x'].values,
               statistics[1][q].loc[emission_mask, 'pos_y'].values,
               color='r',
               s=10.
    )
    # Exclusion
    # theta goes from 0 to 2pi
    theta = np.linspace(0, 2*np.pi, 50)
    # the radius of the circle
    r = config['geometry']['exclusion']['radius']
    # compute x1 and x2
    x1 = r*np.cos(theta)
    x2 = r*np.sin(theta)
    ax3.plot(x1 + config['geometry']['exclusion']['x_pos'], x2 + config['geometry']['exclusion']['y_pos'],
            lw=lw, color='k')
    # Additional options
    ax3.set_xlabel(r'$X\;[\mathrm{m}]$', fontsize=fontsize)
    ax3.set_ylabel(r'$Y\;[\mathrm{m}]$', fontsize=fontsize)
    ax3.tick_params(axis = 'both', which = 'major', labelsize=fontsize)
    ax3.tick_params(axis = 'both', which = 'minor', labelsize=fontsize)
    ax3.title=ax.set_title('Population t = %.f s' %(fd.t[q] / 5.), fontsize=fontsize)
    ax3.set_xlim(0., 40.)
    ax3.set_ylim(4., 16.)
    # -----------------------------------------------------------------------------------------------------
    # The light emission curve
    for j, detector in enumerate(fd.measured.keys()):
        ax4.plot((fd.t / 5.)[:q+1],
             measured[1][detector].values[:q+1] + noise[j, :q+1],
             lw=lw, label=detector)
    #ax1.set_ylim(1e6, 1e11)
    ax4.set_xscale('linear')
    ax4.set_yscale('log')
    ax4.set_xlabel(r'$t\;[$s$]$', fontsize=fontsize)
    ax4.set_ylabel(r'$Unweighted\;Photons$', fontsize=fontsize)
    ax4.tick_params(axis = 'both', which = 'major', labelsize=fontsize, direction='in')
    ax4.tick_params(axis = 'both', which = 'minor', labelsize=fontsize, direction='in')
    ax4.grid(True)
    h, l = ax4.get_legend_handles_labels()
    lgd1 = ax4.legend(h,l, loc=9, bbox_to_anchor=(0.9, +1.),
                      ncol=1, fontsize=fontsize, handlelength=h_length,
                      fancybox=True, frameon=True)
    ax4.set_xlim(fd.t[off_set] / 5., max(fd.t / 5.))
    ax4.set_ylim(2e1,1e5)
    # ----------------------------------------------------------------------------------------------------
    figure.savefig(PICS + '\\Frames\\frame_%d.jpeg' %i, quality=40)
    plt.close(figure)






0it [00:00, ?it/s]




1it [00:02,  2.63s/it]




2it [00:05,  2.71s/it]




3it [00:08,  2.76s/it]




4it [00:10,  2.69s/it]




5it [00:13,  2.66s/it]




6it [00:16,  2.62s/it]




7it [00:18,  2.59s/it]




8it [00:21,  2.58s/it]




9it [00:23,  2.65s/it]




10it [00:26,  2.75s/it]




11it [00:40,  6.01s/it]




12it [00:43,  4.99s/it]




13it [00:45,  4.20s/it]




14it [00:48,  3.73s/it]




15it [00:50,  3.35s/it]




16it [00:53,  3.20s/it]




17it [00:55,  3.00s/it]




18it [00:59,  3.28s/it]




19it [01:02,  3.18s/it]




20it [01:05,  3.01s/it]




21it [01:08,  2.89s/it]




22it [01:10,  2.76s/it]




23it [01:13,  2.75s/it]




24it [01:15,  2.54s/it]




25it [01:17,  2.41s/it]




26it [01:19,  2.30s/it]




27it [01:21,  2.24s/it]




28it [01:23,  2.21s/it]




29it [01:26,  2.50s/it]




30it [01:29,  2.38s/it]




31it [01:31,  2.38s/it]




32it [01:33,  2.44s/it]




33it [01:36,  2.36s/it]




34it [01:38,  2.35s/it]




35it [01:40,  2.30s/it]




3

188it [09:23,  3.19s/it]




189it [09:26,  3.22s/it]




190it [09:30,  3.28s/it]




191it [09:33,  3.22s/it]




192it [09:36,  3.22s/it]




193it [09:39,  3.22s/it]




194it [09:43,  3.29s/it]




195it [09:46,  3.24s/it]




196it [09:49,  3.24s/it]




197it [09:53,  3.39s/it]




198it [09:56,  3.29s/it]




199it [09:59,  3.32s/it]




200it [10:03,  3.45s/it]




201it [10:06,  3.40s/it]




202it [10:09,  3.33s/it]




203it [10:12,  3.27s/it]




204it [10:15,  3.17s/it]




205it [10:18,  3.10s/it]




206it [10:21,  3.06s/it]




207it [10:24,  3.03s/it]




208it [10:27,  3.02s/it]




209it [10:30,  3.07s/it]




210it [10:33,  3.05s/it]




211it [10:36,  3.02s/it]




212it [10:40,  3.12s/it]




213it [10:43,  3.10s/it]




214it [10:46,  3.09s/it]




215it [10:49,  3.22s/it]




216it [10:53,  3.28s/it]




217it [10:56,  3.18s/it]




218it [10:59,  3.13s/it]




219it [11:07,  4.56s/it]




220it [11:10,  4.07s/it]




221it [11:13,  3.75s/it]




222it [11:16, 

374it [20:35,  4.12s/it]




375it [20:39,  4.01s/it]




376it [20:42,  3.82s/it]




377it [20:46,  3.74s/it]




378it [20:49,  3.66s/it]




379it [20:54,  3.89s/it]




380it [20:58,  4.01s/it]




381it [21:02,  4.07s/it]




382it [21:06,  4.16s/it]




383it [21:11,  4.19s/it]




384it [21:15,  4.21s/it]




385it [21:19,  4.32s/it]




386it [21:24,  4.38s/it]




387it [21:29,  4.44s/it]




388it [21:33,  4.47s/it]




389it [21:38,  4.46s/it]




390it [21:41,  4.25s/it]




391it [21:45,  4.06s/it]




392it [21:49,  3.95s/it]




393it [21:52,  3.93s/it]




394it [21:56,  3.93s/it]




395it [22:00,  3.97s/it]




396it [22:05,  4.27s/it]




397it [22:10,  4.23s/it]




398it [22:14,  4.22s/it]




399it [22:18,  4.11s/it]




400it [22:21,  3.99s/it]




401it [22:25,  3.82s/it]




402it [22:28,  3.70s/it]




403it [22:32,  3.61s/it]




404it [22:35,  3.55s/it]




405it [22:38,  3.53s/it]




406it [22:42,  3.52s/it]




407it [22:45,  3.49s/it]




408it [22:49, 

In [24]:
images = []
filenames = np.array([
    PICS + 'Frames\\frame_%d.jpeg' %i
    for i in range(len(fd.t[off_set:]))
])
for filename in filenames[::3]:
    images.append(imageio.imread(filename))
imageio.mimsave(PICS + '%s.gif' %config['water']['model']['name'],
                images, fps=10)